In [1]:
import tensorflow as tf

tf.keras.backend.clear_session()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
K.set_learning_phase(0)

IMG_HEIGHT = 96
IMG_WIDTH = 96
batch_size = 128
num_classes = 2

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    weights="imagenet",
    include_top=False,
    pooling="avg")

model = tf.keras.Sequential([
    base_model,
    Dense(1),
    Activation("sigmoid")
])

model.load_weights('masked-face-image-classification/masked_face_classifier.h5')

for layer in model.layers:
    layer.trainable = False

def freeze_session(session, inputs, keep_var_names=None, output_names=None, clear_devices=True):

    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    
    graph = session.graph
    with graph.as_default():
        tf.identity(inputs, name='image_tensor')
        freeze_var_names = ['image_tensor'] + list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        tf.identity(model.outputs[0], name='classification_scores')
        output_names = ['classification_scores']
        output_names += [v.op.name for v in tf.global_variables()]
        
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

inputs = model.inputs[0]
frozen_graph = freeze_session(K.get_session(), inputs)
tf.train.write_graph(frozen_graph, "./", "masked_face_classifier.pb", as_text=False)

Using TensorFlow backend.
W0617 00:23:34.048005 140064532158208 deprecation.py:506] From /home/indraputramr/venv/lib/python3.5/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0617 00:23:38.927792 140064532158208 deprecation.py:323] From <ipython-input-1-4ed0f98630ea>:54: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0617 00:23:38.928652 140064532158208 deprecation.py:323] From /home/indraputramr/venv/lib/python3.5/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_im

'./masked_face_classifier.pb'